# Wikipedia training

In this tutorial we will:
 - Learn how to train the NMF topic model on English Wikipedia corpus
 - Compare it with LDA model
 - Evaluate results

In [1]:
%load_ext autoreload
%autoreload 2

import itertools
import json
import logging
import numpy as np
import pandas as pd
import scipy.sparse
import smart_open
import time
import os
import psutil
from contextlib import contextmanager
from multiprocessing import Process
from tqdm import tqdm, tqdm_notebook
import joblib

import gensim.downloader as api
from gensim import matutils
from gensim.corpora import MmCorpus, Dictionary
from gensim.models import LdaModel, CoherenceModel
from gensim.models.nmf import Nmf as GensimNmf
from sklearn.decomposition.nmf import NMF as SklearnNmf
from gensim.parsing.preprocessing import preprocess_string

tqdm.pandas()

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

# Preprocessing

### Load wikipedia dump
Let's use `gensim.downloader.api` for that

In [2]:
data = api.load("wiki-english-20171001")
article = next(iter(data))

for section_title, section_text in zip(
    article['section_titles'],
    article['section_texts']
):
    print("Section title: %s" % section_title)
    print("Section text: %s" % section_text[:100])

Section title: Introduction
Section text: 




'''Anarchism''' is a political philosophy that advocates self-governed societies based on volun
Section title: Etymology and terminology
Section text: 

The word ''anarchism'' is composed from the word ''anarchy'' and the suffix ''-ism'', themselves d
Section title: History
Section text: 

===Origins===
Woodcut from a Diggers document by William Everard

The earliest anarchist themes ca
Section title: Anarchist schools of thought
Section text: 
Portrait of philosopher Pierre-Joseph Proudhon (1809–1865) by Gustave Courbet. Proudhon was the pri
Section title: Internal issues and debates
Section text: 
consistent with anarchist values is a controversial subject among anarchists.

Anarchism is a philo
Section title: Topics of interest
Section text: Intersecting and overlapping between various schools of thought, certain topics of interest and inte
Section title: Criticisms
Section text: 
Criticisms of anarchism include moral criticisms and pra

Preprocess and save articles

In [3]:
def save_preprocessed_articles(filename, articles):
    with smart_open(filename, 'w+', encoding="utf8") as writer:
        for article in tqdm_notebook(articles):
            article_text = " ".join(
                " ".join(section)
                for section
                in zip(
                    article['section_titles'],
                    article['section_texts']
                )
            )
            article_text = preprocess_string(article_text)

            writer.write(json.dumps(article_text) + '\n')


def get_preprocessed_articles(filename):
    with smart_open(filename, 'r', encoding="utf8") as reader:
        for line in tqdm_notebook(reader):
            yield json.loads(
                line
            )

In [4]:
SAVE_ARTICLES = False

if SAVE_ARTICLES:
    save_preprocessed_articles('wiki_articles.jsonlines', data)

### Create and save dictionary

In [5]:
SAVE_DICTIONARY = False

if SAVE_DICTIONARY:
    dictionary = Dictionary(get_preprocessed_articles('wiki_articles.jsonlines'))
    dictionary.save('wiki.dict')

### Load and filter dictionary

In [6]:
dictionary = Dictionary.load('wiki.dict')
dictionary.filter_extremes()
dictionary.compactify()

2019-02-02 10:03:37,182 : INFO : loading Dictionary object from wiki.dict
2019-02-02 10:03:38,090 : INFO : loaded wiki.dict
2019-02-02 10:03:43,122 : INFO : discarding 1910146 tokens: [('abdelrahim', 49), ('abstention', 120), ('anarcha', 101), ('anarchica', 40), ('anarchosyndicalist', 20), ('antimilitar', 68), ('arbet', 194), ('archo', 100), ('arkhē', 5), ('autonomedia', 118)]...
2019-02-02 10:03:43,123 : INFO : keeping 100000 tokens which were in no less than 5 and no more than 2462447 (=50.0%) documents
2019-02-02 10:03:44,029 : INFO : resulting dictionary: Dictionary(100000 unique tokens: ['skittl', 'steindachn', 'hackworth', 'rbi', 'gaml']...)


### MmCorpus wrapper
In this way we'll:

- Make sure that documents are shuffled
- Be able to train-test split corpus without rewriting it

In [7]:
class RandomCorpus(MmCorpus):
    def __init__(self, random_seed=42, testset=False, testsize=1000, *args,
                 **kwargs):
        super().__init__(*args, **kwargs)

        random_state = np.random.RandomState(random_seed)
        
        self.indices = random_state.permutation(range(self.num_docs))
        test_nnz = sum(len(self[doc_idx]) for doc_idx in self.indices[:testsize])
        
        if testset:
            self.indices = self.indices[:testsize]
            self.num_docs = testsize
            self.num_nnz = test_nnz
        else:
            self.indices = self.indices[testsize:]
            self.num_docs -= testsize
            self.num_nnz -= test_nnz

    def __iter__(self):
        for doc_id in self.indices:
            yield self[doc_id]

### Create and save corpus

In [8]:
SAVE_CORPUS = False

if SAVE_CORPUS:
    corpus = (
        dictionary.doc2bow(article)
        for article
        in get_preprocessed_articles('wiki_articles.jsonlines')
    )
    
    RandomCorpus.serialize('wiki.mm', corpus)

### Load train and test corpus
Using `RandomCorpus` wrapper

In [9]:
train_corpus = RandomCorpus(
    random_seed=42, testset=False, testsize=2000, fname='wiki.mm'
)
test_corpus = RandomCorpus(
    random_seed=42, testset=True, testsize=2000, fname='wiki.mm'
)

2019-02-02 10:03:44,960 : INFO : loaded corpus index from wiki.mm.index
2019-02-02 10:03:44,961 : INFO : initializing cython corpus reader from wiki.mm
2019-02-02 10:03:44,962 : INFO : accepted corpus with 4924894 documents, 100000 features, 683326444 non-zero entries
2019-02-02 10:03:47,635 : INFO : loaded corpus index from wiki.mm.index
2019-02-02 10:03:47,636 : INFO : initializing cython corpus reader from wiki.mm
2019-02-02 10:03:47,637 : INFO : accepted corpus with 4924894 documents, 100000 features, 683326444 non-zero entries


### Convert corpora to csc and save

In [10]:
SAVE_CSC = False

if SAVE_CSC:
    train_csc = matutils.corpus2csc(train_corpus, len(dictionary))
    scipy.sparse.save_npz('train_csc.npz', train_csc)
    
    test_csc = matutils.corpus2csc(test_corpus, len(dictionary))
    scipy.sparse.save_npz('test_csc.npz', test_csc)

## Metrics

In [11]:
@contextmanager
def measure_ram(output, tick=2):
    def _measure_ram(pid, output, start_memory, tick=5):
        py = psutil.Process(pid)
        with open(output, 'w') as outfile:
            while True:
                memory = py.memory_info().rss - start_memory
                outfile.write("{}\n".format(memory))
                outfile.flush()
                time.sleep(tick)

    pid = os.getpid()
    start_memory = psutil.Process(pid).memory_info().rss
    p = Process(target=_measure_ram, args=(pid, output, start_memory, 5))
    p.start()
    yield
    p.terminate()

def get_train_time_and_ram(func, name):
    memprof_filename = "{}.memprof".format(name)
    
    start = time.time()

    with measure_ram(memprof_filename, 5):
        result = func()        
        
    elapsed_time = pd.to_timedelta(time.time() - start, unit='s').round('s')
    
    memprof_df = pd.read_csv(memprof_filename, squeeze=True)
    
    mean_ram = "{} MB".format(
        memprof_df.mean() // 2**20,
    )
    
    max_ram = "{} MB".format(memprof_df.max() // 2**20)

    return elapsed_time, mean_ram, max_ram, result


def get_tm_metrics(model, test_corpus):
    W = model.get_topics().T
    H = np.zeros((model.num_topics, len(test_corpus)))
    for bow_id, bow in enumerate(test_corpus):
        for topic_id, word_count in model.get_document_topics(bow):
            H[topic_id, bow_id] = word_count

    pred_factors = W.dot(H)
    
    dense_corpus = matutils.corpus2dense(test_corpus, pred_factors.shape[0])

    l2_norm = get_tm_l2_norm(pred_factors, dense_corpus)
    
    pred_factors /= pred_factors.sum(axis=0)
    
    perplexity = get_tm_perplexity(pred_factors, dense_corpus)

    model.normalize = True

    coherence = CoherenceModel(
        model=model,
        corpus=test_corpus,
        coherence='u_mass'
    ).get_coherence()

    model.normalize = False

    return dict(
        perplexity=round(perplexity, 4),
        coherence=round(coherence, 4),
        l2_norm=round(l2_norm, 4),
    )


def get_tm_perplexity(pred_factors, dense_corpus):
    return np.exp(-(np.log(pred_factors, where=pred_factors > 0) * dense_corpus).sum() / dense_corpus.sum())


def get_tm_l2_norm(pred_factors, dense_corpus):
    return np.linalg.norm(dense_corpus - pred_factors)


def get_sklearn_metrics(model, test_corpus):
    W = model.components_.T
    H = model.transform((test_corpus).T).T
    pred_factors = W.dot(H)
    
    l2_norm = np.linalg.norm(test_corpus - pred_factors)
    
    pred_factors /= pred_factors.sum(axis=0)

    perplexity = np.exp(
        -(np.log(pred_factors, where=pred_factors > 0) * test_corpus).sum()
        / test_corpus.sum()
    )

    return dict(
        perplexity=perplexity,
        l2_norm=l2_norm,
    )

Define dataframe in which we'll store metrics

In [12]:
tm_metrics = pd.DataFrame()

### Define common params for the models

In [13]:
params = dict(
    corpus=train_corpus,
    chunksize=2000,
    num_topics=50,
    id2word=dictionary,
    passes=1,
    eval_every=10,
    minimum_probability=0,
    random_state=42,
)

## Training

### Train Gensim NMF and save it
Normalization is turned off to compute metrics correctly

In [14]:
row = dict()
row['model'] = 'gensim_nmf'
row['train_time'], row['mean_ram'], row['max_ram'], nmf = get_train_time_and_ram(
    lambda: GensimNmf(
        normalize=False,
        **params
    ),
    'gensim_nmf',
)

nmf.save('gensim_nmf.model')

2019-02-02 10:04:53,525 : INFO : Loss: 0.04250465036259249
2019-02-02 10:05:31,238 : INFO : Loss: 0.035834409283392266
2019-02-02 10:05:44,738 : INFO : Loss: 0.01666296904388558
2019-02-02 10:06:02,355 : INFO : Loss: 0.014592172927081915
2019-02-02 10:06:24,328 : INFO : Loss: 0.03738619426262098
2019-02-02 10:06:42,237 : INFO : Loss: 0.011548734523320468
2019-02-02 10:06:56,768 : INFO : Loss: 0.005949484242001706
2019-02-02 10:07:05,775 : INFO : Loss: 0.0054164438291086905
2019-02-02 10:07:14,770 : INFO : Loss: 0.005098339670066033
2019-02-02 10:07:24,161 : INFO : Loss: 0.0042822453766947766
2019-02-02 10:07:33,062 : INFO : Loss: 0.004218452750957835
2019-02-02 10:07:43,446 : INFO : Loss: 0.004957368001488409
2019-02-02 10:07:51,063 : INFO : Loss: 0.004023650353364507
2019-02-02 10:08:00,500 : INFO : Loss: 0.004060624865162843
2019-02-02 10:08:08,661 : INFO : Loss: 0.0007936539858723043
2019-02-02 10:08:16,650 : INFO : Loss: 0.002926750080973836
2019-02-02 10:08:27,915 : INFO : Loss: 0

2019-02-02 10:20:29,236 : INFO : Loss: 0.00028238131271271424
2019-02-02 10:20:34,567 : INFO : Loss: 0.0002640478005720415
2019-02-02 10:20:39,655 : INFO : Loss: 0.00022538729252012077
2019-02-02 10:20:44,604 : INFO : Loss: 0.0
2019-02-02 10:20:49,844 : INFO : Loss: 0.0
2019-02-02 10:20:55,181 : INFO : Loss: 0.00024749787367713824
2019-02-02 10:21:00,564 : INFO : Loss: 0.0005803275246304228
2019-02-02 10:21:05,900 : INFO : Loss: 0.00016246434292861256
2019-02-02 10:21:10,822 : INFO : Loss: 0.0003649175874610073
2019-02-02 10:21:16,070 : INFO : Loss: 0.00043936818925609906
2019-02-02 10:21:21,261 : INFO : Loss: 0.0
2019-02-02 10:21:26,508 : INFO : Loss: 0.0001473140708686702
2019-02-02 10:21:31,603 : INFO : Loss: 0.00030760951796811754
2019-02-02 10:21:36,910 : INFO : Loss: 0.0001534249170939211
2019-02-02 10:21:42,224 : INFO : Loss: 0.0002268004146246076
2019-02-02 10:21:47,424 : INFO : Loss: 0.00020589154304862873
2019-02-02 10:21:52,546 : INFO : Loss: 0.00042309497830752803
2019-02-0

### Load Gensim NMF and store metrics

In [15]:
nmf = GensimNmf.load('gensim_nmf.model')
row.update(get_tm_metrics(nmf, test_corpus))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

2019-02-02 10:29:44,278 : INFO : loading Nmf object from gensim_nmf.model
2019-02-02 10:29:44,586 : INFO : loading id2word recursively from gensim_nmf.model.id2word.* with mmap=None
2019-02-02 10:29:44,586 : INFO : loaded gensim_nmf.model
/home/anotherbugmaster/gensim/gensim/matutils.py:503: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)
2019-02-02 10:30:24,320 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2019-02-02 10:30:24,431 : INFO : CorpusAccumulator accumulated stats from 2000 documents


### Train LDA and save it
That's a common model to do Topic Modeling

In [16]:
row = dict()
row['model'] = 'lda'
row['train_time'], row['mean_ram'], row['max_ram'], lda = get_train_time_and_ram(
    lambda: LdaModel(**params),
    'lda',
)
lda.save('lda.model')

2019-02-02 10:30:24,537 : INFO : using symmetric alpha at 0.02
2019-02-02 10:30:24,538 : INFO : using symmetric eta at 0.02
2019-02-02 10:30:24,554 : INFO : using serial LDA version on this node
2019-02-02 10:30:25,076 : INFO : running online (single-pass) LDA training, 50 topics, 1 passes over the supplied corpus of 4922894 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2019-02-02 10:30:25,231 : INFO : PROGRESS: pass 0, at document #2000/4922894
2019-02-02 10:30:27,186 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-02 10:30:27,503 : INFO : topic #36 (0.020): 0.006*"new" + 0.005*"record" + 0.004*"serv" + 0.004*"includ" + 0.003*"yawn" + 0.003*"start" + 0.003*"depress" + 0.003*"word" + 0.003*"théori" + 0.003*"american"
2019-02-02 10:30:27,505 : INFO : topic #19 (0.020): 0.005*"team" + 0.005*"new" + 0.004*"nation" + 0.004*"start" + 0.004*"includ

2019-02-02 10:30:38,765 : INFO : topic diff=0.220294, rho=0.408248
2019-02-02 10:30:38,923 : INFO : PROGRESS: pass 0, at document #14000/4922894
2019-02-02 10:30:40,583 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-02 10:30:40,841 : INFO : topic #39 (0.020): 0.038*"team" + 0.025*"scientist" + 0.025*"coach" + 0.020*"leagu" + 0.015*"place" + 0.012*"player" + 0.012*"folei" + 0.010*"hoar" + 0.009*"yawn" + 0.009*"basketbal"
2019-02-02 10:30:40,842 : INFO : topic #0 (0.020): 0.035*"station" + 0.031*"arsen" + 0.015*"line" + 0.014*"railwai" + 0.012*"museum" + 0.012*"paint" + 0.012*"left" + 0.011*"align" + 0.010*"word" + 0.009*"artist"
2019-02-02 10:30:40,843 : INFO : topic #13 (0.020): 0.020*"sourc" + 0.018*"north" + 0.015*"weekli" + 0.013*"earthworm" + 0.012*"castl" + 0.012*"lagrang" + 0.010*"cotton" + 0.009*"vigour" + 0.008*"vallei" + 0.008*"hormon"
2019-02-02 10:30:40,845 : INFO : topic #45 (0.020): 0.015*"depress" + 0.007*"snake" + 0.006*"stanc" + 0

2019-02-02 10:30:55,470 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-02 10:30:55,731 : INFO : topic #10 (0.020): 0.010*"cdd" + 0.009*"disco" + 0.007*"cancer" + 0.007*"gastrointestin" + 0.007*"acid" + 0.007*"caus" + 0.006*"pathwai" + 0.006*"treat" + 0.006*"proper" + 0.005*"includ"
2019-02-02 10:30:55,733 : INFO : topic #18 (0.020): 0.007*"man" + 0.006*"kill" + 0.005*"charact" + 0.005*"théori" + 0.004*"septemb" + 0.004*"later" + 0.004*"appear" + 0.004*"episod" + 0.004*"storag" + 0.004*"faster"
2019-02-02 10:30:55,734 : INFO : topic #0 (0.020): 0.055*"station" + 0.038*"arsen" + 0.024*"line" + 0.024*"railwai" + 0.016*"paint" + 0.016*"word" + 0.013*"align" + 0.012*"traceabl" + 0.012*"museum" + 0.012*"london"
2019-02-02 10:30:55,735 : INFO : topic #37 (0.020): 0.007*"love" + 0.007*"théori" + 0.006*"place" + 0.006*"rel" + 0.005*"appear" + 0.005*"night" + 0.004*"man" + 0.004*"song" + 0.003*"gestur" + 0.003*"yawn"
2019-02-02 10:30:55,736 : INFO : topic 

2019-02-02 10:31:07,817 : INFO : topic #40 (0.020): 0.072*"unit" + 0.025*"collector" + 0.015*"start" + 0.014*"american" + 0.011*"new" + 0.011*"institut" + 0.009*"scholar" + 0.009*"word" + 0.009*"governor" + 0.008*"professor"
2019-02-02 10:31:07,818 : INFO : topic #25 (0.020): 0.016*"mount" + 0.015*"river" + 0.015*"palmer" + 0.014*"lagrang" + 0.013*"mound" + 0.011*"area" + 0.008*"pcb" + 0.007*"robespierr" + 0.006*"natur" + 0.006*"survei"
2019-02-02 10:31:07,819 : INFO : topic #27 (0.020): 0.045*"questionnair" + 0.015*"dai" + 0.012*"team" + 0.012*"tour" + 0.012*"théori" + 0.010*"rider" + 0.010*"hors" + 0.010*"squatter" + 0.010*"finish" + 0.009*"sebastien"
2019-02-02 10:31:07,821 : INFO : topic #48 (0.020): 0.066*"octob" + 0.062*"januari" + 0.061*"march" + 0.057*"sens" + 0.055*"notion" + 0.053*"judici" + 0.053*"april" + 0.052*"juli" + 0.051*"august" + 0.051*"decatur"
2019-02-02 10:31:07,827 : INFO : topic diff=0.286263, rho=0.229416
2019-02-02 10:31:10,711 : INFO : -11.672 per-word bound,

2019-02-02 10:31:22,477 : INFO : topic #48 (0.020): 0.065*"octob" + 0.063*"notion" + 0.063*"januari" + 0.063*"sens" + 0.060*"judici" + 0.060*"march" + 0.059*"april" + 0.059*"august" + 0.057*"decatur" + 0.056*"juli"
2019-02-02 10:31:22,478 : INFO : topic #21 (0.020): 0.025*"san" + 0.021*"spain" + 0.019*"mexico" + 0.016*"del" + 0.012*"juan" + 0.012*"soviet" + 0.012*"mexican" + 0.011*"plung" + 0.010*"santa" + 0.010*"josé"
2019-02-02 10:31:22,479 : INFO : topic #49 (0.020): 0.028*"india" + 0.022*"incumb" + 0.007*"singh" + 0.006*"peopl" + 0.006*"pakistan" + 0.006*"treeless" + 0.006*"alam" + 0.006*"pradesh" + 0.006*"area" + 0.006*"khalsa"
2019-02-02 10:31:22,485 : INFO : topic diff=0.237375, rho=0.200000
2019-02-02 10:31:22,637 : INFO : PROGRESS: pass 0, at document #52000/4922894
2019-02-02 10:31:24,196 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-02 10:31:24,461 : INFO : topic #21 (0.020): 0.027*"san" + 0.020*"spain" + 0.019*"mexico" + 0.018*"del" 

...

2019-02-02 11:56:28,353 : INFO : topic #40 (0.020): 0.086*"unit" + 0.023*"schuster" + 0.022*"institut" + 0.021*"requir" + 0.020*"collector" + 0.019*"student" + 0.014*"professor" + 0.012*"word" + 0.011*"governor" + 0.011*"degre"
2019-02-02 11:56:28,359 : INFO : topic diff=0.003171, rho=0.020265
2019-02-02 11:56:28,515 : INFO : PROGRESS: pass 0, at document #4872000/4922894
2019-02-02 11:56:29,915 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-02 11:56:30,182 : INFO : topic #41 (0.020): 0.043*"citi" + 0.025*"palmer" + 0.019*"new" + 0.018*"street" + 0.013*"center" + 0.013*"open" + 0.012*"includ" + 0.011*"lobe" + 0.011*"dai" + 0.009*"highli"
2019-02-02 11:56:30,183 : INFO : topic #28 (0.020): 0.037*"build" + 0.030*"hous" + 0.019*"buford" + 0.015*"histor" + 0.012*"linear" + 0.012*"centuri" + 0.012*"silicon" + 0.011*"depress" + 0.011*"constitut" + 0.010*"pistol"
2019-02-02 11:56:30,184 : INFO : topic #49 (0.020): 0.043*"india" + 0.029*"incumb" + 0.013*

2019-02-02 11:56:41,875 : INFO : topic diff=0.003092, rho=0.020240
2019-02-02 11:56:42,031 : INFO : PROGRESS: pass 0, at document #4884000/4922894
2019-02-02 11:56:43,410 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-02 11:56:43,677 : INFO : topic #31 (0.020): 0.050*"fusiform" + 0.027*"scientist" + 0.025*"team" + 0.023*"player" + 0.019*"place" + 0.014*"coach" + 0.013*"leagu" + 0.010*"folei" + 0.010*"yawn" + 0.009*"record"
2019-02-02 11:56:43,678 : INFO : topic #32 (0.020): 0.051*"district" + 0.047*"popolo" + 0.043*"vigour" + 0.035*"tortur" + 0.032*"cotton" + 0.023*"multitud" + 0.022*"adulthood" + 0.021*"area" + 0.019*"cede" + 0.018*"commun"
2019-02-02 11:56:43,679 : INFO : topic #25 (0.020): 0.031*"river" + 0.019*"area" + 0.017*"lagrang" + 0.016*"warmth" + 0.015*"mount" + 0.010*"north" + 0.009*"sourc" + 0.009*"land" + 0.009*"foam" + 0.008*"palmer"
2019-02-02 11:56:43,680 : INFO : topic #9 (0.020): 0.074*"bone" + 0.042*"american" + 0.030*"van" + 

2019-02-02 11:56:54,761 : INFO : topic #25 (0.020): 0.031*"river" + 0.019*"area" + 0.017*"lagrang" + 0.016*"warmth" + 0.015*"mount" + 0.010*"north" + 0.009*"sourc" + 0.009*"land" + 0.009*"palmer" + 0.009*"foam"
2019-02-02 11:56:54,762 : INFO : topic #16 (0.020): 0.055*"king" + 0.030*"princ" + 0.020*"duke" + 0.019*"royal" + 0.018*"idiosyncrat" + 0.018*"queen" + 0.018*"grand" + 0.016*"kingdom" + 0.014*"portugues" + 0.012*"crown"
2019-02-02 11:56:54,763 : INFO : topic #8 (0.020): 0.027*"law" + 0.023*"cortic" + 0.018*"start" + 0.016*"act" + 0.012*"ricardo" + 0.012*"case" + 0.010*"replac" + 0.010*"polaris" + 0.008*"legal" + 0.008*"judaism"
2019-02-02 11:56:54,765 : INFO : topic #36 (0.020): 0.011*"network" + 0.011*"prognosi" + 0.010*"radio" + 0.009*"cytokin" + 0.008*"develop" + 0.008*"digit" + 0.008*"softwar" + 0.007*"uruguayan" + 0.007*"user" + 0.007*"includ"
2019-02-02 11:56:54,766 : INFO : topic #7 (0.020): 0.021*"son" + 0.020*"di" + 0.018*"factor" + 0.016*"yawn" + 0.016*"margin" + 0.014

2019-02-02 11:57:08,352 : INFO : topic #32 (0.020): 0.051*"district" + 0.046*"popolo" + 0.042*"vigour" + 0.036*"tortur" + 0.033*"cotton" + 0.022*"adulthood" + 0.022*"multitud" + 0.021*"area" + 0.019*"cede" + 0.018*"citi"
2019-02-02 11:57:08,353 : INFO : topic #34 (0.020): 0.067*"start" + 0.033*"new" + 0.031*"american" + 0.030*"unionist" + 0.026*"cotton" + 0.020*"year" + 0.015*"california" + 0.013*"washington" + 0.012*"texa" + 0.012*"north"
2019-02-02 11:57:08,354 : INFO : topic #38 (0.020): 0.022*"walter" + 0.011*"aza" + 0.009*"forc" + 0.008*"battalion" + 0.008*"teufel" + 0.007*"till" + 0.007*"empath" + 0.007*"armi" + 0.006*"militari" + 0.006*"govern"
2019-02-02 11:57:08,355 : INFO : topic #16 (0.020): 0.056*"king" + 0.031*"princ" + 0.020*"duke" + 0.019*"royal" + 0.018*"idiosyncrat" + 0.018*"queen" + 0.017*"grand" + 0.016*"kingdom" + 0.014*"portugues" + 0.011*"crown"
2019-02-02 11:57:08,361 : INFO : topic diff=0.003263, rho=0.020187
2019-02-02 11:57:08,515 : INFO : PROGRESS: pass 0, at

2019-02-02 11:57:21,978 : INFO : topic #48 (0.020): 0.077*"octob" + 0.076*"march" + 0.076*"sens" + 0.068*"januari" + 0.068*"notion" + 0.067*"decatur" + 0.067*"juli" + 0.066*"august" + 0.065*"april" + 0.063*"judici"
2019-02-02 11:57:21,979 : INFO : topic #1 (0.020): 0.052*"china" + 0.043*"chilton" + 0.025*"kong" + 0.025*"hong" + 0.021*"korea" + 0.019*"korean" + 0.016*"leah" + 0.015*"sourc" + 0.015*"shirin" + 0.014*"kim"
2019-02-02 11:57:21,980 : INFO : topic #3 (0.020): 0.031*"present" + 0.026*"offic" + 0.025*"nation" + 0.023*"govern" + 0.023*"minist" + 0.022*"member" + 0.019*"serv" + 0.018*"start" + 0.015*"gener" + 0.014*"chickasaw"
2019-02-02 11:57:21,981 : INFO : topic #25 (0.020): 0.031*"river" + 0.019*"area" + 0.017*"lagrang" + 0.017*"warmth" + 0.015*"mount" + 0.010*"north" + 0.009*"sourc" + 0.009*"land" + 0.008*"palmer" + 0.008*"foam"
2019-02-02 11:57:21,987 : INFO : topic diff=0.003423, rho=0.020162
2019-02-02 11:57:22,147 : INFO : PROGRESS: pass 0, at document #4922000/4922894
2

### Load LDA and store metrics

In [17]:
lda = LdaModel.load('lda.model')
row.update(get_tm_metrics(lda, test_corpus))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

2019-02-02 11:57:26,542 : INFO : loading LdaModel object from lda.model
2019-02-02 11:57:26,544 : INFO : loading expElogbeta from lda.model.expElogbeta.npy with mmap=None
2019-02-02 11:57:26,549 : INFO : setting ignored attribute id2word to None
2019-02-02 11:57:26,550 : INFO : setting ignored attribute dispatcher to None
2019-02-02 11:57:26,550 : INFO : setting ignored attribute state to None
2019-02-02 11:57:26,551 : INFO : loaded lda.model
2019-02-02 11:57:26,551 : INFO : loading LdaState object from lda.model.state
2019-02-02 11:57:26,644 : INFO : loaded lda.model.state
/home/anotherbugmaster/gensim/gensim/matutils.py:503: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)
2019-02-02 11:57:34,745 : INFO : CorpusAccumulator accumulated stats from 100

### Train Sklearn NMF and store metrics

In [18]:
row = dict()
row['model'] = 'sklearn_nmf'
sklearn_nmf = SklearnNmf(n_components=50, tol=1e-2, random_state=42)
row['train_time'], row['mean_ram'], row['max_ram'], sklearn_nmf = get_train_time_and_ram(
    lambda: sklearn_nmf.fit(scipy.sparse.load_npz('train_csc.npz').T),
    'sklearn_nmf',
)

joblib.dump(sklearn_nmf, 'sklearn_nmf.joblib')

['sklearn_nmf.joblib']

### Load Sklearn NMF and store metrics

In [19]:
sklearn_nmf = joblib.load('sklearn_nmf.joblib')
row.update(get_sklearn_metrics(
    sklearn_nmf, scipy.sparse.load_npz('test_csc.npz').toarray(),
))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

In [20]:
tm_metrics.replace(np.nan, '-', inplace=True)

## Results

In [21]:
tm_metrics

,coherence,l2_norm,max_ram,mean_ram,model,perplexity,train_time
0,-2.9077,1986.291000,170 MB,136.0 MB,gensim_nmf,3817.239000,00:25:55
1,-2.5286,2273.643000,145 MB,145.0 MB,lda,4701.976000,01:27:02
2,-,1987.280856,16917 MB,10709.0 MB,sklearn_nmf,3943.036299,00:51:14


As we can see, NMF can be significantly faster than LDA without sacrificing quality of topics too much (or not sacrificing at all)

Moreover, NMF can be very flexible on RAM usage due to sparsity option, which leaves only small amount of elements in inner matrices.